In [ ]:
!pip install git+https://github.com/gautamchitnis/cocoapi.git@cocodataset-master#subdirectory=PythonAPI

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir("/content/drive/MyDrive/SB")
!ls

In [ ]:
device = "cuda"
EPOCHS = 6
BATCH_SIZE = 8
NUM_WORKERS = 2
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
TRAIN_IMAGE_PATH = '/content/drive/MyDrive/SB/ears/train/'
TRAIN_MASK_PATH = '/content/drive/MyDrive/SB/ears/annotations/segmentation/train/'
TRAIN_YOLO_PATH = '/content/drive/MyDrive/SB/ears/annotations/detection/train_YOLO_format/'

TEST_IMAGE_PATH = '/content/drive/MyDrive/SB/ears/test/'
TEST_MASK_PATH = '/content/drive/MyDrive/SB/ears/annotations/segmentation/test/'
TEST_YOLO_PATH = '/content/drive/MyDrive/SB/ears/annotations/detection/test_YOLO_format/'



In [ ]:
import os
import numpy as np
import cv2
from torch.utils.data import Dataset
from PIL import Image
from matplotlib import pyplot as plt
%matplotlib notebook
%matplotlib inline

class EarDataset(Dataset):
    def __init__(self, image_dir, mask_dir, yolo_dir, transform):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.yolo_dir = yolo_dir
        self.transform = transform
        self.imgs = list(sorted(os.listdir(image_dir)))
        self.masks = list(sorted(os.listdir(mask_dir)))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, item):
        img_path = os.path.join(self.image_dir, self.imgs[item])
        mask_path = os.path.join(self.mask_dir, self.masks[item])
        yolo_path = os.path.join(self.yolo_dir, self.imgs[item].replace("png", "txt"))
        yolo_boxes = []
        with open(yolo_path) as file:
            yolo_boxes.append(file.readline().split(" ")[1:])
        box = yolo_boxes[0]       
        image = Image.open(img_path).convert("RGB")
        #image = np.array(image)
        #img = self.preprocess.pre(image)
        #image = Image.fromarray(img)
        mask = Image.open(mask_path)
        
        mask = np.array(mask)[None, :, :] == 255
        
        


        # get bounding box coordinates for each mask
        
        
        boxes = []
        for box in yolo_boxes:
            xmin = int(box[0])
            xmax = int(box[0]) + int(box[2])
            ymin = int(box[1])
            ymax = int(box[1]) + int(box[3])
            boxes.append([xmin, ymin, xmax, ymax])

        num_objs = len(boxes)
        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(mask, dtype=torch.uint8)
        image_id = torch.tensor([item])
        
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transform is not None:
            image, target= self.transform(image, target)
            

           
        return image, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
import torchvision
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection import MaskRCNN


def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    bbone = torchvision.models.mobilenet_v2(pretrained=True).features
            
    bbone.out_channels = 1280

    
    anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                        aspect_ratios=((0.5, 1.0, 2.0),))

    
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                    output_size=7,
                                                    sampling_ratio=2)

    mask_roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                          output_size=14,
                                                          sampling_ratio=2)

    model = MaskRCNN(bbone,
                          num_classes=2,
                          rpn_anchor_generator=anchor_generator,
                          box_roi_pool=roi_pooler,
                          mask_roi_pool=mask_roi_pooler)
    

    return model

In [ ]:
# Download TorchVision repo to use some files from
# references/detection
!git clone https://github.com/pytorch/vision.git
#!cd vision

!cp vision/references/detection/utils.py ./
!cp vision/references/detection/coco_eval.py ./
!cp vision/references/detection/engine.py ./
!cp vision/references/detection/coco_utils.py ./

fatal: destination path 'vision' already exists and is not an empty directory.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


In [ ]:

!cp /content/drive/MyDrive/SB/preprocess.py ./
!cp /content/drive/MyDrive/SB/transforms.py ./
import transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.HistogramEqualization())
    #transforms.append(T.ImageSharpening())
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

def store_predictions(model, test_data_loader):
    data_iter = iter(test_data_loader)

    model.eval()
    i = 0
    for images, targets in data_iter:
        images = list(image for image in images)
        outputs = model(images)
        outputs = outputs[0]["masks"]
        masks = [mask.to(torch.device('cpu')).detach().numpy()[0] for mask in outputs]
        mask = np.zeros(masks[0].shape)
        for m in masks:
            mask = np.logical_or(mask, m > 0.5)
        mask = mask.astype("uint8")
        mask *= 255
        im = Image.fromarray(mask, mode='L').convert('1')
        im.save("/content/drive/MyDrive/SB/ears/predictions"f"/{i+1:04d}"+".png")
        i += 1
                  

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
cp: '/content/drive/MyDrive/SB/preprocess.py' and './preprocess.py' are the same file
cp: '/content/drive/MyDrive/SB/transforms.py' and './transforms.py' are the same file


In [ ]:
from engine import train_one_epoch, evaluate
import utils
import torch


def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # our dataset has two classes only - background and person
    num_classes = 2
    # use our dataset and defined transformations
    dataset = EarDataset(TRAIN_IMAGE_PATH, TRAIN_MASK_PATH, TRAIN_YOLO_PATH, get_transform(train=True))
    dataset_test = EarDataset(TEST_IMAGE_PATH, TEST_MASK_PATH, TEST_YOLO_PATH, get_transform(train=False))

    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=False, num_workers=4,
        collate_fn=utils.collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=utils.collate_fn)

    # get the model using our helper function
    model = get_model_instance_segmentation(num_classes)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.Adam(params, lr=1e-5, weight_decay=1e-2)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    # let's train it for 10 epochs
    num_epochs = 10

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        #store_predictions(model, data_loader_test)

        #save the model
        torch.save(model.state_dict(), "model3_weights_epoch{}.pth".format(epoch))

    print("That's it!")
main()

In [ ]:
import torch

def iou_compute(p, gt):
    # Computes Intersection Over Union (IOU)
    if len(p) == 0:
        return 0

    intersection = np.logical_and(p, gt)
    union = np.logical_or(p, gt)

    iou = np.sum(intersection) / np.sum(union)

    return iou 

def dice_score(p, gt):
    num_correct = (p == gt).sum()
    num_pixels = torch.numel()
     

def F1(p, gt):
    true_positive = (p == 255.0 and gt == 255.0).sum()
    false_positive = (p == 255.0 and gt == 0.0).sum()
    false_negative = (p == 0.0 and gt == 255.0).sum()
    presision = true_positive / (true_positive + false_positive)
    recall true_positive / (true_positive + false_negative)

    return 2 * recall * precision / (recall + precision)


In [ ]:
from matplotlib import pyplot as plt
import torch
dataset_test = EarDataset(TEST_IMAGE_PATH, TEST_MASK_PATH, TEST_YOLO_PATH, get_transform(train=False), preprocess)
data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=2,
        collate_fn=utils.collate_fn)

# get the model using our helper function
num_classes=2
model = get_model_instance_segmentation(num_classes)
model.load_state_dict(torch.load("model3_weights_epoch4.pth"))
# move model to the right device
model.to(device)
store_predictions(model, data_loader_test)
     
length = len(dataset_test)
# put the model in evaluation mode
model.eval()

iou_all = 0
for i, item in enumerate(dataset_test):
    with torch.no_grad():
        prediction = model([item[0].to(device)])
    iou = iou_compute(prediction[0]["masks"].cpu().numpy(), item[1]['masks'].cpu().numpy())
    plt.imsave   
    print(iou) 
    iou_all += iou

print(iou_all/length)

#Image.fromarray(prediction[0]['masks'][0, 0].mul(255).byte().cpu().numpy())
#for img, _ in enumerate(dataset_test):
#  model.eval()
#  with torch.no_grad():
##    prediction = model([img.to(device)])
#   Image.fromarray(prediction[0]['masks'][0, 0].mul(255).byte().cpu().numpy())




In [ ]:
def run_evaluation():

    iou_arr = []
    model = get_model_instance_segmentation(num_classes=2)
    model.load_state_dict(torch.load("model3_weights_epoch3.pth"))
    
    

    for i in range(250):
        # Read an image
        img = cv2.imread("/content/drive/MyDrive/SB/ears/predictions"+f"/{i+1:04d}"+".png")
        gt = cv2.imread("/content/drive/MyDrive/SB/ears/annotations/segmentation/test"+f"/{i+1:04d}"+".png")
        
        iou = iou_compute(img, gt)
        iou_arr.append(iou)

    miou = np.average(iou_arr)
    print("\n")
    print("Average IOU:", f"{miou:.2%}")
    print("\n")

In [ ]:
run_evaluation()